In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time


In [2]:
raw_df=pd.read_csv(r"C:\Users\amith\Downloads\ofqual_details_20250127.csv\ofqual_details_20250127.csv")
# Filter rows where status is "Available to learners"
new_df = raw_df[raw_df['status'] == "Available to learners"]


In [4]:
from urllib.parse import urlparse
import pandas as pd


# Create a new column for the domain using netloc
new_df['domain'] = new_df['specification'].apply(lambda x: urlparse(x).netloc if pd.notnull(x) else '')

# Filter rows where domain matches
nocn_df = new_df[new_df['domain'] == "www.lantra.co.uk"]

nocn_df


C:\Users\amith\AppData\Local\Temp\ipykernel_19024\2632889648.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['domain'] = new_df['specification'].apply(lambda x: urlparse(x).netloc if pd.notnull(x) else '')


,qualification_type,qualification_level,qualification_number,assessment_methods,sector_subject_area,awarding_organisation,grading_type,total_credits,funding_in_england,status,...,european_qualification_level,ofqual_id,guided_learning_hours,total_qualification_time,specification,specialisms,optional_routes,grading_scale,end-point_assessment_standard,domain
5383,Vocationally-Related Qualification,Level 1,600/2182/2,"Portfolio of Evidence, Practical Demonstration...",Animal care and veterinary science,/organisations/RN5152,Pass/Fail,11.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 2,600/2182/2,99.0,114.0,https://www.lantra.co.uk/qualification-specifi...,NaN,NaN,NaN,NaN,www.lantra.co.uk
5388,Vocationally-Related Qualification,Level 2,501/1738/5,Multiple Choice Examination,Agriculture,/organisations/RN5152,Pass/Fail,3.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 3,501/1738/5,23.0,32.0,https://www.lantra.co.uk/qualification-specifi...,NaN,NaN,NaN,NaN,www.lantra.co.uk
5390,Vocationally-Related Qualification,Level 1,603/2226/3,Multiple Choice Examination,Horticulture and forestry,/organisations/RN5152,Pass/Fail,2.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 2,603/2226/3,12.0,18.0,http://www.lantra.co.uk/awards/qualification-s...,NaN,NaN,NaN,NaN,www.lantra.co.uk
5391,Vocationally-Related Qualification,Level 2,603/2166/0,"Portfolio of Evidence, Practical Demonstration...",Building and construction,/organisations/RN5152,Pass/Fail,16.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 3,603/2166/0,125.0,160.0,http://www.lantra.co.uk/system/files/Qual%20Sp...,NaN,The Lantra Awards Level 2 Certificate in Highw...,NaN,NaN,www.lantra.co.uk
5392,Vocationally-Related Qualification,Level 3,603/2168/4,"Portfolio of Evidence, Practical Demonstration...",Building and construction,/organisations/RN5152,Pass/Fail,13.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 4,603/2168/4,119.0,130.0,http://www.lantra.co.uk/system/files/Qual%20Sp...,NaN,The Lantra Awards Level 3 Certificate in Highw...,NaN,NaN,www.lantra.co.uk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5500,Vocationally-Related Qualification,Level 2,600/5700/2,"Oral Examination, Practical Examination",Horticulture and forestry,/organisations/RN5152,Pass/Fail,1.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 3,600/5700/2,8.0,12.0,https://www.lantra.co.uk/qualification-specifi...,NaN,NaN,NaN,NaN,www.lantra.co.uk
5501,Vocationally-Related Qualification,Level 3,600/5711/7,"Oral Examination, Practical Examination",Horticulture and forestry,/organisations/RN5152,Pass/Fail,3.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 4,600/5711/7,16.0,26.0,https://www.lantra.co.uk/awards/qualification-...,NaN,NaN,NaN,NaN,www.lantra.co.uk
5502,Vocationally-Related Qualification,Level 3,600/5713/0,"Oral Examination, Practical Examination",Horticulture and forestry,/organisations/RN5152,Pass/Fail,3.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 4,600/5713/0,23.0,31.0,https://www.lantra.co.uk/awards/qualification-...,NaN,NaN,NaN,NaN,www.lantra.co.uk
5504,Occupational Qualification,Level 2,603/5725/3,"Portfolio of Evidence, Practical Demonstration...",Building and construction,/organisations/RN5152,Pass/Fail,45.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 3,603/5725/3,150.0,450.0,https://www.lantra.co.uk/qualification-specifi...,NaN,NaN,NaN,NaN,www.lantra.co.uk


In [7]:
nocn_df.iloc[1]['specification']

'https://www.lantra.co.uk/qualification-specifications'

Extract URLS from searching ofqual ids

In [6]:
import concurrent.futures

def scrape_links_for_id(id_value):
    base_url = "https://www.nocn.org.uk/search-results/?filter=1&query="
    search_url = base_url + str(id_value)
    print(f"Scraping URL: {search_url}")
    
    try:
        response = requests.get(search_url, timeout=15)
        if response.status_code != 200:
            print(f"Error: status code {response.status_code} for {search_url}")
            return []
        
        soup = BeautifulSoup(response.text, "html.parser")
        # Find the table with class "table__primary"
        table = soup.find("table", class_="table__primary")
        if not table:
            print(f"No table found on page: {search_url}")
            return []
        
        # Extract all links from the table using the <a> tag with the specific class
        links = []
        for a_tag in table.find_all("a", class_="table__primary__body__td__link"):
            href = a_tag.get("href")
            if href:
                # Convert relative URL to absolute if needed
                if href.startswith("/"):
                    href = "https://www.nocn.org.uk" + href
                links.append(href)
        return links
    except Exception as e:
        print(f"Exception occurred while processing {search_url}: {e}")
        return []

all_links = []

ids_to_scrape = nocn_df["ofqual_id"].tolist()

# Use ThreadPoolExecutor to run the scraping concurrently
max_workers = 20  # Adjust the number of threads as needed

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit tasks for each ID
    future_to_id = {executor.submit(scrape_links_for_id, id_value): id_value for id_value in ids_to_scrape}
    
    # As tasks complete, collect their results
    for future in concurrent.futures.as_completed(future_to_id):
        id_value = future_to_id[future]
        try:
            links = future.result()
            if links:
                all_links.extend(links)
        except Exception as e:
            print(f"Error processing ID {id_value}: {e}")

# Remove duplicates if needed
all_links = list(set(all_links))

# Dump the extracted links into an Excel file
out_df = pd.DataFrame({"Link": all_links})
out_df.to_excel("extracted_links(nocn).xlsx", index=False)
print(f"Extraction complete. {len(all_links)} links saved to extracted_links(nocn).xlsx")


Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=501/0430/5
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=601/6278/8
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=500/4259/2
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=500/4254/3
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=500/5534/3
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=601/0157/X
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=500/7522/6
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=500/7523/8
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=600/1020/4
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=600/0720/5
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=601/0620/7
Scraping URL: https://www.nocn.org.uk/search-results/?filter=1&query=600/3161/X
Scraping URL: https://www.nocn.org.uk/se

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
from concurrent.futures import ThreadPoolExecutor, as_completed

def download_pdf_from_url(pdf_url, save_dir):
    """Download a PDF from a single URL and save it."""
    try:
        response = requests.get(pdf_url, stream=True)
        response.raise_for_status()
        
        # Extract filename from URL
        filename = os.path.basename(urllib.parse.urlparse(pdf_url).path)
        if not filename.lower().endswith('.pdf'):
            filename += '.pdf'
            
        save_path = os.path.join(save_dir, filename)
        
        # Save the file
        with open(save_path, 'wb') as f:
            f.write(response.content)
        
        return f"Downloaded: {filename}"

    except Exception as e:
        return f"Failed to download {pdf_url}: {str(e)}"

def scrape_pdfs_from_page(url, save_dir):
    """Scrape PDF links from a page and download them."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        pdf_links = soup.find_all('a', href=True)
        
        pdf_urls = set()
        for link in pdf_links:
            href = link['href']
            if href.lower().endswith('.pdf'):
                # Handle relative URLs
                pdf_url = urllib.parse.urljoin(url, href)
                pdf_urls.add(pdf_url)
        
        # Download PDFs from the found URLs
        for pdf_url in pdf_urls:
            print(f"Downloading from: {pdf_url}")
            download_pdf_from_url(pdf_url, save_dir)
        
        return f"Scraped and downloaded from: {url}"

    except Exception as e:
        return f"Failed to scrape {url}: {str(e)}"

def download_pdfs(df, url_column, save_dir=r"C:\Users\amith\Kenpath\OFQUAL\nocn_pdfs", max_workers=15):
    """Download PDFs from URLs in DataFrame using threading."""
    os.makedirs(save_dir, exist_ok=True)
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(scrape_pdfs_from_page, row[url_column], save_dir): row[url_column]
            for _, row in df.iterrows()
        }
        
        for future in as_completed(futures):
            print(future.result())

# Read the Excel file
pdf_df = pd.read_excel(r"C:\Users\amith\Kenpath\zavmo-api\notebooks\ofqual\nocn_links.xlsx")

# Run the downloader
download_pdfs(pdf_df, 'Link', max_workers=15)  # Adjust workers based on internet speed




Scraped and downloaded from: https://www.nocn.org.uk/products/qualifications/610-0071-x-nocn-entry-level-3-award-in-energy-efficiency-and-sustainability/
Scraped and downloaded from: https://www.nocn.org.uk/products/qualifications/25090-603-5467-7-nocn-level-2-nvq-diploma-in-wood-occupations-construction-heritage-architectural-joinery/
Scraped and downloaded from: https://www.nocn.org.uk/products/qualifications/20952-603-3919-6-nocn-level-2-certificate-in-construction-multi-skills/
Scraped and downloaded from: https://www.nocn.org.uk/products/qualifications/610-4331-x-nocn-level-3-diploma-in-low-carbon-heating-technician/
Scraped and downloaded from: https://www.nocn.org.uk/products/qualifications/603-0068-1-nocn-level-5-award-in-productivity-improvement-practice/
Scraped and downloaded from: https://www.nocn.org.uk/products/qualifications/18487-601-4999-1-nocn-level-2-nvq-certificate-in-plant-operations-construction-excavating/
Scraped and downloaded from: https://www.nocn.org.uk/pr

In [1]:
import os
import hashlib

def get_file_hash(file_path):
    """Generate MD5 hash for a file to check for duplicates."""
    hash_md5 = hashlib.md5()
    with open(file_path, 'rb') as f:
        # Read the file in chunks to avoid memory overload with large files
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def delete_duplicates_in_folder(folder_path):
    """Delete duplicate PDFs in a folder based on file content."""
    file_hashes = {}
    duplicates = []
    
    # Traverse through the folder
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                file_path = os.path.join(root, file)
                file_hash = get_file_hash(file_path)
                
                # If the file hash is already seen, it's a duplicate
                if file_hash in file_hashes:
                    duplicates.append(file_path)
                    print(f"Duplicate found: {file_path}")
                else:
                    file_hashes[file_hash] = file_path
    
    # Delete duplicate files
    for duplicate in duplicates:
        try:
            os.remove(duplicate)
            print(f"Deleted duplicate: {duplicate}")
        except Exception as e:
            print(f"Failed to delete {duplicate}: {str(e)}")

# Usage example
folder_path = r"C:\Users\amith\Kenpath\OFQUAL\nocn_pdfs"
delete_duplicates_in_folder(folder_path)
